# Text Analysis


#### The idea of this exercise to perform simple text analysis, a popular concept used in many cutting-edge applications. Also, known as Text Mining - the idea is to retrieve high-quality information from the text. Some of the text mining tasks are: text categorization, text clustering, concept/entity extraction, sentiment analysis, document summarization etc

In [ ]:
from pyspark import SparkContext
sc = SparkContext()

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .getOrCreate()

In [ ]:
# Load the text file in zipped format, yes that's possible!
t = sc.textFile('data/test.ft.txt.bz2')

In [ ]:
t.take(2)

#### Stopwords: The list of most frequenty used words in a specific language. Stopwords do not offer any useful information about a chunk of text, so we generally remove them from the text before progressing further¶

In [ ]:
# Execute this cell to download the list of English stopwords
import urllib.request as urllib
urllib.urlretrieve ("https://raw.githubusercontent.com/stanfordnlp/CoreNLP/master/data/edu/stanford/nlp/patterns/surface/stopwords.txt", "stopwords.txt")

In [ ]:
stopwords = sc.textFile("stopwords.txt").collect()

In [ ]:
# Check the number of partitions
t.getNumPartitions()

In [ ]:
# Increase the number of partitions
t = t.repartition(10)

In [ ]:
t.getNumPartitions() # Check again

In [ ]:
# Split the text into 'tokens' (individual words) by whitespace
traw = ....

In [ ]:
# Discard the first token(word) and take rest
tdata = traw.map(lambda x: x[1:])

In [ ]:
# Create a function which tries to eliminate all the special characters in tokens(words)
# Also, only take words which have length more than 2!
# Hint: Use regex, the module in python is re
# Input: x -> list of words/tokens
# Outout: list of words/tokens with length more than 2 and without any special characters
import re
def replace_special_chars(x):
    out = []
    # YOUR CODE HERE
    return out

In [ ]:
t_semi_clean = tdata.map(replace_special_chars)
t_semi_clean.take(10)

In [ ]:
# Create a function that would make the tokens(words) lowercase and then check if it's a stopword or not.
# If stopword, then discard it
# Input: x -> list of words/tokens
# Outout: list of words/tokens without stopwords
def remove_sw(x):
    out = []
    # YOUR CODE HERE
    return out

In [ ]:
t_clean = t_semi_clean.map(remove_sw)
t_clean.take(10)

In [ ]:
tfinal = t_clean.zipWithIndex()  # add a unique id for the records

In [ ]:
inputdf = spark.createDataFrame(tfinal, ["words", "id"])  # another way of creating a dataframe

In [ ]:
inputdf.show()

In [ ]:
inputdf.cache()  # store in memory for quicker operations, Spark feature!

### Term Frequency (TF): The number of times a specific word occurs in a record

To convert text into word counts, we use the CountVectorizer
https://spark.apache.org/docs/2.1.0/ml-features.html#countvectorizer

PySpark docs:
https://spark.apache.org/docs/latest/api/python/pyspark.ml.html#pyspark.ml.feature.CountVectorizer

In [ ]:
from pyspark.ml.feature import CountVectorizer
# Parameters:
# inputCol: "words"
# outputCol: "rawFeatures"
# vocabSize: 1000
# minDF: 2
cv = .....

In [ ]:
# 'Fit' the model to the input data frame (inputdf) with 'cv'
cvmodel = .....

cvmodel here is a 'CountVectorizerModel', all the functions listed here: https://spark.apache.org/docs/latest/api/python/pyspark.ml.html#pyspark.ml.feature.CountVectorizerModel

In [ ]:
vocab = cvmodel.vocabulary  # Get the vocabulary list out

In [ ]:
# Broadcast the variable to all the workers (Only suitable for static data which would be required in tasks)
vocab_broadcast = sc.broadcast(vocab)

In [ ]:
# 'Transform' the inputdf with the cvmodel
featurizedData = ......

In [ ]:
featurizedData.show()

### Inverse Document Frequency (IDF): How important is a specific word in the whole corpus

https://spark.apache.org/docs/latest/api/python/pyspark.ml.html#pyspark.ml.feature.IDF

In [ ]:
from pyspark.ml.feature import IDF
# Parameters:
# inputCol: "rawFeatures"
# outputCol: "features"
idf = .......

In [ ]:
# fit the model with 'featurizedData'
idfModel = .......

idfModel here is an 'IDFModel', read its functions here: https://spark.apache.org/docs/latest/api/python/pyspark.ml.html#pyspark.ml.feature.IDFModel

In [ ]:
# Transform 'featurizedData' with idfModel
tfidfData = ...... 
# We get TFIDF data above

In [ ]:
tfidfData.select('rawFeatures').take(1)

In [ ]:
tfidfData.select('features').take(1)

### Latent Dirichlet allocation (LDA) is a topic model which infers topics from a collection of text documents. LDA can be thought of as a clustering algorithm where Topics correspond to cluster centers, and documents correspond to examples (rows) in a dataset.

https://spark.apache.org/docs/latest/mllib-clustering.html#latent-dirichlet-allocation-lda

https://spark.apache.org/docs/latest/api/python/pyspark.mllib.html#pyspark.mllib.clustering.LDA

In [ ]:
from pyspark.ml.clustering import LDA
# Group the words into 10 different topics
# Parameters:
# k: 10
# featuresCol: "features"
lda = ......

In [ ]:
# fit the model with 'tfidfData'
ldamodel = .......

https://spark.apache.org/docs/latest/api/python/pyspark.mllib.html#pyspark.mllib.clustering.LDAModel

In [ ]:
# describe the topics from the model
ldatopics = ........

In [ ]:
ldatopics.show()

In [ ]:
def indexToWord(x):
    vocab_local = vocab_broadcast.value  # Read the variable that we broadcasted earlier
    res = []
    for i in x[1]:
        res.append(vocab_local[i])
    return (x[0], res)

In [ ]:
topicsRDD = ldatopics.rdd.map(indexToWord)

In [ ]:
# convert topicsRDD to dataframe passing the names of the columns ['topic', 'word'] and show the contents
